In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import imageio.v3 as iio
%load_ext autoreload
%autoreload 2
from time import sleep

In [2]:
genus = pd.read_csv("data/genus.csv")
genus.head()

,Genus,Value,Distribution
0,Apostasia,1066,"Distribution: Tropical Asia, Malesia, Australia"
1,Neuwiedia,1067,"Distribution: S-China, Indochina, Malesia from..."
2,Duckeella,1068,Distribution: N-South America
3,Pogonia,1069,"Distribution: Temperate E-Asia (4), North Amer..."
4,Cleistesiopsis,1070,Distribution: USA


In [3]:
data = pd.read_csv("data/orchids.csv")
data["SubSpecie"].fillna("", inplace=True)
data.Genus_Info.fillna("", inplace=True)


In [4]:

data["Variant"] = ""
data.head()

,UID,Specie,SubSpecie,Distribution,href,Genus,Genus_Info,Value,Variant
0,0,0,0,0,0,0,0,0,
1,39213,Apostasia elliptica,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,
2,39214,Apostasia fogangica,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,
3,39215,Apostasia latifolia,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",['https://www.worldplants.de/fileadmin/Modules...,Apostasia,,1066,
4,39216,Apostasia nuda,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,


In [28]:
columns = ["UID","Specie","SubSpecie","Variant","Distribution","href","Genus","Genus_Info","Value"]
data_var = pd.DataFrame(columns= columns)
#data_var
#data_var.to_csv("data/orchids_var.csv", index=False, header=True)

In [6]:
data.head()

,UID,Specie,SubSpecie,Distribution,href,Genus,Genus_Info,Value,Variant
0,0,0,0,0,0,0,0,0,
1,39213,Apostasia elliptica,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,
2,39214,Apostasia fogangica,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,
3,39215,Apostasia latifolia,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",['https://www.worldplants.de/fileadmin/Modules...,Apostasia,,1066,
4,39216,Apostasia nuda,,"[['peninsular Malaysia', 'Sumatra', 'Borneo', ...",[],Apostasia,,1066,


In [7]:
#df = data.copy()
#df["SubSpecie"].fillna("", inplace=True)
#df.Genus_Info.fillna("", inplace=True)
#df["Variant"] = ""
#df = df[["UID","Specie","SubSpecie","Variant","Distribution","href","Genus","Genus_Info","Value"]]
#df.head()

In [8]:
base_url = "https://www.worldplants.de/world-orchids/orchid-list"

In [9]:
def connection_to_webpage(base_url,value):
# we generate the url
    url = base_url + "#g-"+value
    browser = webdriver.Chrome()

    browser.get(url)
    wait = WebDriverWait(browser,20)
    html = browser.page_source

    return browser

In [10]:
def get_var_species(browser):
    sleep(3)
    orchid_sub = browser.find_elements(By.CSS_SELECTOR, ".type-variant")
    name_sub = []
    dist_sub = []
    href_sub = []
    h = []
    data_sub = []
    distribution = []
    for i in orchid_sub:
        u_id =int(i.get_attribute("id").split("-")[1])
        n = i.find_element(By.CSS_SELECTOR, ".latName").text
        name_sub.append(u_id)
        tmp = i.find_elements(By.CSS_SELECTOR, ".distCountry")
        for j in tmp:
            dist_sub.append([j.text])
        distribution.append(dist_sub)
        dist_sub = []
        image = i.find_elements(By.CSS_SELECTOR, "div.imageBox a")
        for j in image:
            tmp =j.get_attribute("href")
            if tmp not in h:
                h.append(tmp)
        href_sub.append(h)
        data_sub.append([u_id,n,distribution,h])
        h = []
    return data_sub, name_sub

In [11]:
def get_species_sub_species(name_sub,data,data_sub):
    j = 0
    k = 0
    spe = ""
    for i in name_sub:
        while k == 0:
            if (i-j) in data["UID"].values:
                spe = data[data["UID"] == (i-j)]["Specie"].unique()
                data_sub.loc[data_sub["UID"]==i,"Specie"] = spe
                k = 1
            j += 1
        print("out of while", i)
        j = 0
        k = 0    
    return data_sub

In [12]:
def create_images(data):
    for i in data.index:
        m = data["href"][i]
        name = data["Specie"][i]
        sub = data["SubSpecie"][i]
        var = data["Variant"][i]
        if len(m) != 0:
            for j,k in enumerate(m):             
                tmp = iio.imread(k)
                iio.imwrite(f"images/{name}-{var}-{j}.jpg",tmp)
                sleep(1)
    return "Images created successfully"

In [13]:
def save_data(data):
    data = data[["UID","Specie","SubSpecie","Variant","Distribution","href","Genus","Genus_Info","Value"]]
    data.to_csv('data/orchids_var.csv',mode='a', index=False, header=False)
    return "Data appended successfully."

In [14]:
def make_df(data,columns,*extra):
    df = pd.DataFrame(data, columns =columns)
    for i in extra:
        df[i] = ""
    sleep(1)
    return df

In [15]:
def add_columns(g, g_d,v,data):
    data["Genus"] = g
    data["Genus_Info"] = g_d
    data["Value"] = v
    sleep(1)
    return data

In [16]:
daa =

SyntaxError: invalid syntax (4036101673.py, line 1)

In [ ]:
#data = data[["UID","Specie","SubSpecie","Variant","Distribution","href","Genus","Genus_Info","Value"]]
#data.to_csv('data/orchids2.csv', index=False, header=True)

<h2> The main program

In [29]:
for i in range(352,len(genus)):
    v = genus["Value"][i]
    g = genus["Genus"][i]
    d = genus["Distribution"]
    url = base_url + "#g-"+str(v)
    browser = webdriver.Chrome()
    browser.get(url)
    html = browser.page_source
    data_var, name_var= get_var_species(browser)
    wait = WebDriverWait(browser,20)
    data_var = make_df(data_var, ["UID","Variant","Distribution","href"], "Specie", "SubSpecie")
    #df = pd.concat([data,data_var],ignore_index = True)
    if data_var.shape[0] > 0:
        data_var = get_species_sub_species(name_var,data,data_var)
        data_var = add_columns(g, d,v,data_var)
        save_data(data_var)
        create_images(data_var)
        sleep(5)
    browser.close()
    print(f"row {i}")

out of while 57344
row 352
out of while 57357
out of while 57371
out of while 57394
row 353
row 354
row 355
row 356
row 357
row 358
out of while 57531
out of while 57548
out of while 57569
row 359
row 360
out of while 57578
row 361
row 362
row 363
row 364
row 365
row 366
row 367
row 368
row 369
out of while 57659
row 370
row 371
row 372
row 373
row 374
row 375
out of while 57725
row 376
out of while 57733
row 377
row 378
row 379
row 380
out of while 57775
row 381
out of while 57831
row 382
row 383
row 384
row 385
row 386
row 387
row 388
row 389
row 390
row 391
row 392
row 393
out of while 57956
row 394
row 395
row 396
row 397
row 398
row 399
row 400
out of while 58054
out of while 58060
row 401
out of while 58070
out of while 58106
row 402
row 403
row 404
row 405
row 406
row 407
row 408
row 409
row 410
out of while 58282
out of while 58283
out of while 58284
out of while 58285
out of while 58287
row 411
row 412
out of while 58358
out of while 58371
out of while 58372
out of while 58373

In [25]:
dat =
i = 125
v = genus["Value"][i]
g = genus["Genus"][i]
d = genus["Distribution"]
url = base_url + "#g-"+str(v)
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
data_var, name_var= get_var_species(browser)
wait = WebDriverWait(browser,20)
data_var = make_df(data_var, ["UID","Variant","Distribution","href"], "Specie", "SubSpecie")
#df = pd.concat([data,data_var],ignore_index = True)
if data_var.shape[0] > 0:
    data_var = get_species_sub_species(name_var,data,data_var)
    data_var = add_columns(g, d,v,data_var)
    save_data(data_var)
    create_images(data_var)
    sleep(5)
browser.close()
print(f"row {i}")

row 125
